In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def json_to_df(json_path):
  with open(json_path, 'r') as f:
    df = pd.read_json(f)
  return df

In [ ]:
test_path = '/content/drive/MyDrive/NQ_test_top100_all.json'
test_rerank_path_1 = '/content/drive/MyDrive/NQ_test_top100_reranked_bert_1.json'
test_rerank_path_2 = '/content/drive/MyDrive/NQ_test_top100_reranked_bert_2.json'

test_df = json_to_df(test_path)

test_rerank_1 = json_to_df(test_rerank_path_1)
test_rerank_2 = json_to_df(test_rerank_path_2)
test_rerank = pd.concat([test_rerank_1, test_rerank_2], ignore_index=True)

In [ ]:
dev_qe_path = '/content/drive/MyDrive/NQ_dev_top100_all.json'
dev_qe_rerank_t5_path_1 = '/content/drive/MyDrive/NQ_dev_top100_qe_reranked_t5_1.json'
dev_qe_rerank_t5_path_2 = '/content/drive/MyDrive/NQ_dev_top100_qe_reranked_t5_2.json'

dev_qe_df = json_to_df(dev_qe_path)

dev_qe_rerank_t5_1 = json_to_df(dev_qe_rerank_t5_path_1)
dev_qe_rerank_t5_2 = json_to_df(dev_qe_rerank_t5_path_2)
dev_qe_rerank_t5_df = pd.concat([dev_qe_rerank_t5_1, dev_qe_rerank_t5_2], ignore_index=True)

In [ ]:
dev_rerank_t5_path_1 = '/content/drive/MyDrive/NQ_dev_top100_reranked_t5_1.json'
dev_rerank_t5_path_2 = '/content/drive/MyDrive/NQ_dev_top100_reranked_t5_2.json'

dev_rerank_t5_1 = json_to_df(dev_rerank_t5_path_1)
dev_rerank_t5_2 = json_to_df(dev_rerank_t5_path_2)
dev_rerank_t5_df = pd.concat([dev_rerank_t5_1, dev_rerank_t5_2], ignore_index=True)

In [ ]:
def calculate_precision(df_json=pd.DataFrame, k=100):
  precision = 0
  for i in range(len(df_json)): # iterate per row
    cnt = 0
    pos_lst = [1 for x in df_json['ctxs'][i] if x['has_answer']]
    for j in range(k): # iterate per ctxs
      if df_json['ctxs'][i][j]['has_answer']:
        cnt += 1
    if len(pos_lst):
      curr_precision = cnt / len(pos_lst)
    else:
      curr_precision = 0
    precision += curr_precision
  return precision / len(df_json)

In [ ]:
def calculate_mrr(df_json=pd.DataFrame, k=100):
  mrr = 0
  for i in range(len(df_json)): # iterate per row
    for j in range(k): # iterate per ctxs
      if df_json['ctxs'][i][j]['has_answer']:
        mrr += 1 / (j + 1) # (j+1)-th rank
        break
  return mrr / len(df_json)

In [ ]:
def calculate_map(df_json=pd.DataFrame, k=100):
  map = 0
  for i in range(len(df_json)): # iterate per row
    cnt = 0
    curr_precision = 0
    for j in range(k): # iterate per ctxs
      if df_json['ctxs'][i][j]['has_answer']:
        cnt += 1
        curr_precision += cnt / (j + 1)
    if cnt:
      map += curr_precision / cnt
  return map / len(df_json)

In [ ]:
def calculate_ndcg(df_json=pd.DataFrame, k=100):
  ndcg = 0
  for i in range(len(df_json)): # iterate per row
    dcg = 0
    cnt = 0
    ideal_dcg = 1
    for j in range(k): # iterate per ctxs
      if df_json['ctxs'][i][j]['has_answer']:
        dcg += 1 / np.log2(j + 2)
        ideal_dcg += 1 / np.log2(cnt + 2)
        cnt += 1
    if ideal_dcg or dcg:
      ndcg += dcg / ideal_dcg
  return ndcg / len(df_json)

In [ ]:
print(calculate_precision(dev_qe_df, 100))
print(calculate_mrr(dev_qe_df, 100))
print(calculate_map(dev_qe_df, 100))
print(calculate_ndcg(dev_qe_df, 100))